In [43]:
import torch
import torch.nn as nn
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import PIL

In [44]:
## Checking the CUDA
if torch.cuda.is_available():
    print("CUDA is available! PyTorch can use the GPU.")
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch will use the CPU.")

CUDA is available! PyTorch can use the GPU.
Current GPU: NVIDIA GeForce RTX 4060 Laptop GPU


## Loading The Data

In [45]:
data_dir = '/root/cw2_fold/Data/images_original'

data_transforms = {
transforms.Compose([
       transforms.ToTensor()
       
    ]),
}


In [46]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)  # List of class folders
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}  # Map class names to indices
        self.images = self._load_images()  # List of (image_path, class_index) tuples

    def _load_images(self):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                images.append((img_path, self.class_to_idx[cls]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')  # Open image and convert to RGB
        if self.transform:
            image = self.transform(image)
        return image, label

In [47]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [48]:
full_dataset = CustomImageDataset(root_dir=data_dir, transform=train_transforms)

# Define the sizes for train, validation, and test sets
train_size = int(0.7 * len(full_dataset))  # 70% for training
val_size = int(0.15 * len(full_dataset))   # 15% for validation
test_size = len(full_dataset) - train_size - val_size  # Remaining 15% for testing

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Apply different transformations for validation and test sets
val_dataset.dataset.transform = val_test_transforms
test_dataset.dataset.transform = val_test_transforms

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [49]:
# Example usage
for images, labels in train_loader:
    print(images.shape, labels)
    break

torch.Size([32, 3, 224, 224]) tensor([4, 4, 8, 3, 0, 4, 1, 8, 5, 2, 4, 6, 5, 7, 9, 6, 5, 1, 5, 6, 2, 8, 4, 8,
        9, 9, 5, 2, 3, 0, 0, 1])


In [50]:
input_size=(180,180)
hidden_size1=64
hidden_size2=32
output_size=10

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.layer1=nn.Linear(input_size,hidden_size1)
    self.layer2=nn.Linear(hidden_size1,hidden_size2)
    self.layer3=nn.Linear(hidden_size2,output_size)
    self.relu=nn.ReLU()
  def forward(self,x):
    x=x.flatten(start_dim=1)
    x=self.layer1(x)
    x=self.relu(x)
    x=self.layer2(x)
    x=self.relu(x)
    x=self.layer3(x)

    return x